## TYTANチュートリアル（連立方程式を解く）

2023年5月6日

ビネクラ安田

出典：[量子アニーリング（D-wave）で連立方程式を解いてみた](https://vigne-cla.com/21-18/)


### QUBOでの考え方

基本の条件設定**「ｎ個の量子ビットからｍ個を１にする」**から考える。

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

これはつまり、2個の量子ビットを足した値が2になるということ。それぞれの量子ビットは [0, 1] のどちらかを取る。合計値が2であればエネルギーは最小で、合計値が2からずれるとエネルギーが高くなる。

ここで、量子ビット名を [x, y, z] に変えて、それぞれに係数 [5, -1, 2] をかける。

```
H = (5*x - y + 2*z - 7)**2
```

これで方程式<font color="red">「5x – y + 2z = 7」</font>が完成。「5x – y + 2z」の部分が7のときにエネルギーは最小で、7からずれるとエネルギーが高くなる。

このような方程式を複数設定する。

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

## 解が 0, 1 だけからなる連立方程式

$$
\left\{
\begin{array}{ll}
5x - y +2z &=  7  \\
-3x +4y + z &= -2 \\
x -2y -4z &= -3
\end{array}
\right.
$$

$$
\begin{eqnarray*}
x, y, z = 1, 0, 1
\end{eqnarray*}
$$

解が０または１であることが分かっていることを前提とする。解が０と１だけなので [x, y, z] をそのまま量子ビットで表せる。

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [2]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
x = symbols('x')
y = symbols('y')
z = symbols('z')

#連立方程式の設定
H = 0
H += ( 5*x -  y +2*z - 7)**2
H += (-3*x +4*y +  z + 2)**2
H += (   x -2*y -4*z + 3)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#確認
for r in result:
    print(r)

offset
62
[{'x': 1.0, 'y': 0.0, 'z': 1.0}, -62.0, 500]


解が [1, 0, 1] と出た。オフセットと解のエネルギーに着目すると、**マイナスオフセットが理論上の最小エネルギー**で、それに一致するエネルギーの解が得られている。この解は３つの方程式を完全に満たしていることを意味する（数学の問題なので完全に満たす解が存在するはず）。

## 解が 0, 1, 2, 3 だけからなる連立方程式

$$
\left\{
\begin{array}{ll}
x + y + z &=  6 \\
2x +3y -2z &= 11 \\
3x - y + z &=  4
\end{array}
\right.
$$

$$
\begin{eqnarray*}
x, y, z = 2, 3, 1
\end{eqnarray*}
$$

解が０～３の整数であることが分かっていることを前提とする。この場合、[x, y, z] をそのまま量子ビットで表すことができない。そこで [x0, x1] という量子ビットを用意して、「x = 2\*x0 + x1」のように２進数表現（2bit）する。x0が2<sup>1</sup>の桁、x1が2<sup>0</sup>の桁。

In [4]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
x0 = symbols('x0')
x1 = symbols('x1')

y0 = symbols('y0')
y1 = symbols('y1')

z0 = symbols('z0')
z1 = symbols('z1')

#x,y,zを2進数（2bit）で表す
x = 2*x0 + x1
y = 2*y0 + y1
z = 2*z0 + z1

#連立方程式の設定
H = 0
H += (  x +  y +  z -  6)**2
H += (2*x +3*y -2*z - 11)**2
H += (3*x -  y +  z -  4)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#確認
for r in result:
    print(r)
    #2進数から戻して確認
    x0, x1 = list(r[0].values())[:2]
    y0, y1 = list(r[0].values())[2:4]
    z0, z1 = list(r[0].values())[4:]
    print(f'x = {2*x0 + x1}')
    print(f'y = {2*y0 + y1}')
    print(f'z = {2*z0 + z1}')

offset
173
[{'x0': 1.0, 'x1': 0.0, 'y0': 1.0, 'y1': 1.0, 'z0': 0.0, 'z1': 1.0}, -173.0, 263]
x = 2.0
y = 3.0
z = 1.0
[{'x0': 1.0, 'x1': 0.0, 'y0': 1.0, 'y1': 0.0, 'z0': 0.0, 'z1': 0.0}, -168.0, 237]
x = 2.0
y = 2.0
z = 0.0


10進数に戻して確認するのが少し手間だが、x, y, z = 2, 3, 1 が得られた。

## 解が 0～255 の整数からなる連立方程式

$$
\left\{
\begin{array}{ll}
10x +14y +4z &= 5120 \\
 9x +12y +2z &= 4230 \\
 7x + 5y +2z &= 2360
\end{array}
\right.
$$

$$
\begin{eqnarray*}
x, y, z = 130, 230, 150
\end{eqnarray*}
$$

解が0～255の整数であることが分かっていることを前提とする。お察しの通り、[x0, x1, x2, x3, x4, x5, x6, x7] という量子ビットを用意して、「x = 128\*x0 + 64\*x1 + 32\*x2 + 16\*x3 + 8\*x4 + 4\*x5 + 2\*x6 + x7」という２進数表現（8bit）をする。

In [5]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
x0 = symbols('x0')
x1 = symbols('x1')
x2 = symbols('x2')
x3 = symbols('x3')
x4 = symbols('x4')
x5 = symbols('x5')
x6 = symbols('x6')
x7 = symbols('x7')

y0 = symbols('y0')
y1 = symbols('y1')
y2 = symbols('y2')
y3 = symbols('y3')
y4 = symbols('y4')
y5 = symbols('y5')
y6 = symbols('y6')
y7 = symbols('y7')

z0 = symbols('z0')
z1 = symbols('z1')
z2 = symbols('z2')
z3 = symbols('z3')
z4 = symbols('z4')
z5 = symbols('z5')
z6 = symbols('z6')
z7 = symbols('z7')

#x,y,zを2進数（8bit）で表す
x = 128*x0 + 64*x1 + 32*x2 + 16*x3 + 8*x4 + 4*x5 + 2*x6 + x7
y = 128*y0 + 64*y1 + 32*y2 + 16*y3 + 8*y4 + 4*y5 + 2*y6 + y7
z = 128*z0 + 64*z1 + 32*z2 + 16*z3 + 8*z4 + 4*z5 + 2*z6 + z7

#連立方程式の設定
H = 0
H += (10*x +14*y +4*z - 5120)**2
H += ( 9*x +12*y +2*z - 4230)**2
H += ( 7*x + 5*y +2*z - 2360)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位5件
for r in result[:5]:
    print(r)
    #2進数から戻して確認
    x0,x1,x2,x3,x4,x5,x6,x7 = list(r[0].values())[:8]
    y0,y1,y2,y3,y4,y5,y6,y7 = list(r[0].values())[8:16]
    z0,z1,z2,z3,z4,z5,z6,z7 = list(r[0].values())[16:]
    print(f'x = {128*x0+64*x1+32*x2+16*x3+8*x4+4*x5+2*x6+x7}')
    print(f'y = {128*y0+64*y1+32*y2+16*y3+8*y4+4*y5+2*y6+y7}')
    print(f'z = {128*z0+64*z1+32*z2+16*z3+8*z4+4*z5+2*z6+z7}')

offset
49676900
[{'x0': 1.0, 'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 0.0, 'x6': 1.0, 'x7': 0.0, 'y0': 1.0, 'y1': 1.0, 'y2': 1.0, 'y3': 0.0, 'y4': 0.0, 'y5': 1.0, 'y6': 1.0, 'y7': 0.0, 'z0': 1.0, 'z1': 0.0, 'z2': 0.0, 'z3': 1.0, 'z4': 0.0, 'z5': 1.0, 'z6': 1.0, 'z7': 0.0}, -49676900.0, 10]
x = 130.0
y = 230.0
z = 150.0
[{'x0': 1.0, 'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 0.0, 'x6': 1.0, 'x7': 1.0, 'y0': 1.0, 'y1': 1.0, 'y2': 1.0, 'y3': 0.0, 'y4': 0.0, 'y5': 1.0, 'y6': 1.0, 'y7': 0.0, 'z0': 1.0, 'z1': 0.0, 'z2': 0.0, 'z3': 1.0, 'z4': 0.0, 'z5': 0.0, 'z6': 1.0, 'z7': 1.0}, -49676886.0, 10]
x = 131.0
y = 230.0
z = 147.0
[{'x0': 1.0, 'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 0.0, 'x6': 0.0, 'x7': 1.0, 'y0': 1.0, 'y1': 1.0, 'y2': 1.0, 'y3': 0.0, 'y4': 0.0, 'y5': 1.0, 'y6': 1.0, 'y7': 0.0, 'z0': 1.0, 'z1': 0.0, 'z2': 0.0, 'z3': 1.0, 'z4': 1.0, 'z5': 0.0, 'z6': 0.0, 'z7': 1.0}, -49676886.0, 7]
x = 129.0
y = 230.0
z = 153.0
[{'x0': 1.0, 'x1': 0.0, 'x2': 0.0, 'x3': 0.

結局、解が0～255の整数という制限の下、3元の連立方程式に24量子ビットを使った。